In [1]:
from Game import *

Version:  0.3.07


## Symmetric Blackjack (i.e. no house)

- shuffle one deck
- each player is dealt 2 cards - one is face up one face down
- 2 players, both equivalent
- move "hit" and "stand"
- if "hit"
    - dealt another card face up - bust - hand score > 21(end of game)
    - other player's turn
- if "stand"
    - end of your turn
 
- end when both players stand
- whoever has the highest hand score wins  (ties as well)
- hand score calculated


In [2]:
def valid_moves(state,player):
    return ["hit","stand"]

In [3]:
def initial_state():

    # State
    # deck
    # hand player 1
    # hand player 2
    # player 1 standing
    # player 2 standing
    # last card player 1
    # last card player 2

    deck=makedeck(1)
    hand1=deck.deal(2)
    hand2=deck.deal(2) 
    standing1=False
    standing2=False    
    last_card1=None
    last_card2=None

    state=deck,hand1,hand2,standing1,standing2

    return state

In [4]:
def score(hand):

    total=0
    has_ace=False
    for card in hand:
        if card.rank==1:
            has_ace=True
            
        if card.rank>10:
            total=total+10
        else:
            total=total+card.rank

    if has_ace:
        if total+10<=21:
            total=total+10
    
    return total
    

In [5]:
score([Card(1,'C'),Card(1,'C'),Card(8,'C'),Card(2,'C')])

12

In [6]:
deck,hand1,hand2,standing1,standing2=initial_state()
print(hand1,score(hand1))
print(hand2,score(hand2))

4H JC 14
4D 9S 13


In [23]:
def win_status(state,player):
    deck,hand1,hand2,standing1,standing2=state

    score1=score(hand1)
    score2=score(hand2)
    visible_score1=score(hand1[1:])
    visible_score2=score(hand2[1:])

    if player==1 and visible_score1>21:
        return "lose"

    if player==2 and visible_score2>21:
        return "lose"

    
    if standing1 and standing2:
        if player==1:

            if score1>21 and score2>21:
                return "tie"
            if score1>21:
                return "lose"
            if score2>21:
                return "win"
                
            
            if score1>score2:
                return 'win'
            elif score1==score2:
                return 'stalemate'
            else:
                return 'lose'

        elif player==2:

            if score1>21 and score2>21:
                return "tie"
            if score1>21:
                return "win"
            if score2>21:
                return "lose"


            
            if score1>score2:
                return 'lose'
            elif score1==score2:
                return 'stalemate'
            else:
                return 'win'
        else:
            raise ValueError("You can't get there from here.")



    

In [24]:
def state_to_observation(state,player):
    
    deck,hand1,hand2,standing1,standing2=state

    if player==1:
        observation=hand1,hand2[1:],standing1,standing2
    else:
        observation=hand2,hand1[1:],standing2,standing1
        

    return observation  # what an agent actually sees

In [25]:
def update_state(state,player,move):
    deck,hand1,hand2,standing1,standing2=state

    if move=='stand':
        if player==1:
            standing1=True

        if player==2:
            standing2=True
            
    elif move=='hit':
        card=deck.deal(1)[0]  # deal one card and take the [0th] element of the list
        if player==1:
            hand1+=[card]
        if player==2:
            hand2+=[card]
        
    else:
        raise ValueError("You can't get there from here.")

    new_state=deck,hand1,hand2,standing1,standing2
    return new_state


In [54]:
def show_state(observation,player):
    my_hand,your_hand,me_standing,you_standing=observation
    
    print(f"My (player {player}) hand {my_hand} with visible score {score(my_hand[1:])} and total score {score(my_hand)}.")
    if you_standing:
        print(f"Other hand {your_hand} showing with visible score {score(your_hand)} and Standing")    
    else:
        print(f"Other hand {your_hand} with visible score {score(your_hand)} .")    



In [55]:
def repeat_move(state,player,move):
    deck,hand1,hand2,standing1,standing2=state

    if player==1 and standing2:
        return True

    if player==2 and standing1:
        return True

    return False

what specifies a card

- rank 1-13
- suit H , C, S, D

- jokers


In [56]:
card=Card(13,'C')

In [57]:
card.rank

13

In [58]:
card.suit

'C'

In [59]:
print(makedeck(1))

10H 8C JS AC 4S 9H 10C QH 6H AS 8S JH 2D 9D KS 10D 7C 4C 5S 7H 8D 3S 4H 2S QD 5D QS 7D 6D 8H 5C 4D 10S QC 3C KC 6C KH JD 9S 5H JC 9C 3D AD AH 6S 2C 7S 3H 2H KD


In [60]:
print(makedeck(2))

2D 6H 10C 10C JH KD 3S 5H 9D 10S 10D 5C 9C 9S AC 3S 2D QS 9C 2C KH 2H QH AH 10D 7S 2C 4C AD KH 7H 9H 6C 6S JS JC KC 7C JC AD 9H 4H JD 4C 8D 8S 5D 4D QD AS AS 6C 6D JH 4H QS 5S 10H 3H 3C AC 6H JS KS 8C 4S 8H 7S 10H 2S 6S 4S 8D 3H 3C 3D 2S 8S QH 9S 5H AH 10S JD QC 5D 7D 4D 3D KC 7H 8C 2H QD 9D 5C KD 6D 7C 7D QC KS 8H 5S


In [61]:
deck=makedeck(1)
len(deck)

52

In [62]:
hand=deck.deal(2)

In [63]:
hand

['AS', 'KH']

In [64]:
len(deck)

50

In [65]:
def random_move(observation,player):    
    moves=valid_moves(observation,player)
    return random.choice(moves)

random_agent=Agent(random_move)


In [66]:
g=Game(number_of_games=1)
g.run(random_agent,random_agent)
g.report()   

====
Game  1
My (player 1) hand 10H KS with visible score 10 and total score 20.
Other hand ['2C'] with visible score 2 .
Player 1 moves hit
My (player 2) hand QS 2C with visible score 2 and total score 12.
Other hand ['KS', '6D'] with visible score 16 .
Player 2 moves stand
My (player 1) hand 10H KS 6D with visible score 16 and total score 26.
Other hand ['2C'] showing with visible score 2 and Standing
Player 1 moves hit
My (player 1) hand 10H KS 6D 4H with visible score 20 and total score 30.
Other hand ['2C'] showing with visible score 2 and Standing
Player 1 moves hit
My (player 1) hand 10H KS 6D 4H 9C with visible score 29 and total score 39.
Other hand ['2C'] showing with visible score 2 and Standing
Player  2 won.
Total number of games:  1
Player 1 (random) Winning 0.00 percent
Player 2 (random) Winning 100.00 percent
Tie 0.00 percent
